## 06-02-01. cuVS를 활용한 GPU 최적화 CAGRA 인덱스 구축 및 검색

In [1]:
import cudf
print("cudf:", cudf.__version__)

cudf: 25.10.00


In [2]:
!pip install -q "cuvs-cu12==25.10.*" --extra-index-url=https://pypi.nvidia.com

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.0/840.0 MB 37.8 MB/s eta 0:00:00


In [5]:
# !pip install -q faiss-gpu-cu12

In [4]:
%%time
import numpy as  np
import cupy as  cp
from cuvs.neighbors import cagra

# 데이터셋 생성 및 GPU로 이동
data  =  cp.random.random((10000, 128), dtype=cp.float32) # 10,000개의 128차원 벡터

# CAGRA 인덱스 파라미터 설정
index_params  = cagra.IndexParams(
metric='sqeuclidean', # 거리 측정 방식: 제곱 유클리드 거리
intermediate_graph_degree=128,
graph_degree=64,
build_algo='ivf_pq'  # 그래프 구축 알고리즘: IVF-PQ
)

# CAGRA 인덱스 생성 및 구축
index  = cagra.build(index_params, data)

# 쿼리 벡터 생성 및 GPU로 이동
queries  =  cp.random.random((5, 128), dtype=cp.float32) # 5개의 128차원 쿼리 벡터

# 검색 파라미터 설정
search_params  = cagra.SearchParams()

# 검색 수행
k  =  10  # 상위 10개의 근접 이웃 검색
distances, neighbors  = cagra.search(index=index, queries=queries, k=k, search_params=search_params)

# 결과 출력 (GPU 메모리에서 CPU 메모리로 복사)
print("이웃 인덱스:\n", cp.asnumpy(neighbors))
print("거리:\n", cp.asnumpy(distances))

이웃 인덱스:
 [[1771 6306    2 9864 3588  889 2638 4406 5525 8799]
 [2026 3667 4694  374 2203  243   53  539 3008  561]
 [6272 3338 4723 8795 2579 9816 1771 3845 4362 3639]
 [8226 9451 9947 7638 5489 5267 3252 6289 3635 2055]
 [4421 5285  571 6730 6427 4143 9239 4603 3644 3318]]
거리:
 [[12.947222  13.457176  13.700697  13.757486  13.939711  13.97297
  13.984667  14.030489  14.045476  14.095133 ]
 [13.551305  14.4824505 14.553976  14.615876  14.918081  15.463015
  15.46752   15.615895  15.620458  15.696445 ]
 [12.660004  13.1896305 13.764275  13.945929  14.14349   14.235136
  14.458668  14.509226  14.533918  14.629697 ]
 [14.931477  15.057591  15.272687  15.412807  15.62591   15.678381
  15.6944065 15.696399  15.765748  15.78781  ]
 [14.615175  15.10644   15.157601  15.269627  15.315754  15.323326
  15.421413  15.426461  15.429567  15.565826 ]]
CPU times: user 1.21 s, sys: 852 ms, total: 2.06 s
Wall time: 4.55 s
